<a href="https://colab.research.google.com/github/abhaysrivastav/finetuning-methods/blob/main/Chapter3_Demo1_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translation from English to Spanish using Flan-T5 and Helsinki-NLP/opus-100 Dataset


## Introduction
In this notebook, we will use the Flan-T5 model to perform translation from English to Spanish. We will use the "Helsinki-NLP/opus-100" dataset from Hugging Face, specifically the en-es subset, to train and evaluate our translation model.


In [23]:
!pip install --upgrade transformers tensorflow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [2]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Loading the Dataset

In [3]:

from datasets import load_dataset

# Load the Helsinki-NLP/opus-100 dataset
dataset = load_dataset('Helsinki-NLP/opus-100', 'en-es')
print(dataset['train'][0])


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/237k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/238k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}


## Data Preprocessing

In [4]:
# Preprocess the dataset for input into the model
def preprocess_data(examples):
    inputs = [f'Translate from English to Spanish: {example["en"]}' for example in examples['translation']]
    targets = [example['es'] for example in examples['translation']]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]

    # For decoder inputs
    decoder_inputs = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["decoder_input_ids"] = decoder_inputs["input_ids"]

    return model_inputs

# Apply preprocessing to the dataset
train_dataset = dataset['train'].select(range(30000)).map(preprocess_data, batched=True)
test_dataset = dataset['test'].map(preprocess_data, batched=True)

print(train_dataset[0])


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}, 'input_ids': [30355, 15, 45, 1566, 12, 5093, 10, 94, 47, 8, 23778, 16, 270, 6, 24, 31, 7, 125, 410, 34, 55, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [6343, 49, 106, 10381

In [5]:
print(type(train_dataset))
print(type(test_dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


## Freezing the Model

In [6]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [6]:
for param in model.shared.parameters():
    param.requires_grad = False

# Freeze the encoder
for param in model.encoder.parameters():
    param.requires_grad = False

# Freeze the decoder
for param in model.decoder.parameters():
    param.requires_grad = False

In [7]:
def params_info(model):
    total_params = 0
    trainable_params = 0
    for param in model.parameters():
        num = param.numel()
        total_params += num
        if param.requires_grad:
            trainable_params += num
    non_trainable_params = total_params - trainable_params

    def size_in_mb(param_count):
        return param_count * 4 / 1024 / 1024  # 4 bytes per param for float32

    print(f"Total params: {total_params:,} ({size_in_mb(total_params):.2f} MB)")
    print(f"Trainable params: {trainable_params:,} ({size_in_mb(trainable_params):.2f} MB)")
    print(f"Non-trainable params: {non_trainable_params:,} ({size_in_mb(non_trainable_params):.2f} MB)")

# Usage
params_info(model)

Total params: 247,577,856 (944.43 MB)
Trainable params: 24,674,304 (94.12 MB)
Non-trainable params: 222,903,552 (850.31 MB)



### Important Considerations in Transfer Learning

1. **Freezing the LLM Layer:** In transfer learning, it's important to freeze the pre-trained language model layer to retain the knowledge it has already acquired and to avoid overfitting. This allows the model to leverage its pre-trained capabilities while focusing on learning the new task-specific nuances.

2. **Loss Function with `from_logits=True`:** When fine-tuning language models from Hugging Face, it's crucial to use the loss function with `from_logits=True`. This is because these models do not apply softmax to their outputs, and using `from_logits=True` ensures that the loss is computed correctly.


## Model Training

In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,      # should be Hugging Face Dataset, not tf.data.Dataset
    eval_dataset=test_dataset,        # should be Hugging Face Dataset, not tf.data.Dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1238658092.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
500,43.414200
1000,42.633900
1500,42.014200
2000,40.982100
2500,40.534500
3000,39.584900
3500,39.103700
4000,38.747200
4500,38.294100
5000,37.632600


TrainOutput(global_step=11250, training_loss=37.811771875, metrics={'train_runtime': 2404.8162, 'train_samples_per_second': 37.425, 'train_steps_per_second': 4.678, 'total_flos': 1.540704043008e+16, 'train_loss': 37.811771875, 'epoch': 3.0})

## Performing Translation

In [17]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 35.517913818359375, 'eval_runtime': 45.3411, 'eval_samples_per_second': 44.11, 'eval_steps_per_second': 5.514, 'epoch': 3.0}


In [18]:
model.save_pretrained("./finetuned-flan-t5")
tokenizer.save_pretrained("./finetuned-flan-t5")

('./finetuned-flan-t5/tokenizer_config.json',
 './finetuned-flan-t5/special_tokens_map.json',
 './finetuned-flan-t5/spiece.model',
 './finetuned-flan-t5/added_tokens.json',
 './finetuned-flan-t5/tokenizer.json')

In [20]:
input_text = "Translate from English to Spanish: How are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# Move input tensors to the same device as the model
device = next(model.parameters()).device  # Get model device
for k in inputs:
    inputs[k] = inputs[k].to(device)

outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Como estás?



## Conclusion
In this notebook, we used the Flan-T5 model to perform translation from English to Spanish using the Helsinki-NLP/opus-100 dataset. We preprocessed the dataset, fine-tuned the model while freezing the LLM layer, and performed translations. We manually validated the translations to assess the quality of the model's performance. The results demonstrate the effectiveness of the Flan-T5 model for translation tasks.
